In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Aprendizaje no supervisado parte 2 - agrupamiento

El agrupamiento (*clustering*) consiste en forma grupos de ejemplos similares de acuerdo a alguna medida de semejanza prefijada o disimilitud (distancia), como la distancia Euclídea.

<img width="60%" src='figures/clustering.png'/>

En esta sección, exploraremos la tarea básica de agrupamiento en algunos datasets sintéticos y del mundo real.

Estas son algunas aplicaciones bastante comunes de los algoritmos de *clustering*:

- Compresión para reducción de datos.
- Resumir los datos como un paso de preprocesamiento para los sistemas de recomendación.
- Agrupar noticias *web* relacionadas (p.ej. Google News) y resultados de búsquedas *web*.
- Realizar perfiles de clientes en estrategias de marketing.

Vamos a empezar creando un dataset sintético simple de dos dimensiones:

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(random_state=42)
X.shape

In [ ]:
plt.scatter(X[:, 0], X[:, 1]);

En el scatter anterior, podemos ver tres grupos separados de datos y nos gustaría recuperarlos utilizando agrupamiento (algo así como "descubrir" las etiquetas de clase, que ya damos por sentadas en la tarea de clasificación).

Incluso si los grupos son obvios en los datos, es difícil encontrarlos cuando estos datos están en un espacio de alta dimensionalidad, que no podemos visualizar en un único histograma o scatterplot.

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=20)

Podemos obtener las etiquetas de los datos o llamando al método ``fit`` y después accediendo al atributo ``labels_`` del estimador KMeans, o llamando a ``fit_predict`` (que aplica los dos pasos seguidos). En cualquier caso, el resultado contiene el identificador del grupo al que asignamos cada punto.

In [ ]:
labels = kmeans.fit_predict(X)

In [ ]:
labels

In [ ]:
print("¿Hemos acertado en todas las etiquetas?", np.all(y == labels))
print("¿En cuantas hemos fallado?", np.sum(y != labels))

Vamos a visualizar lo que hemos obtenido

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=labels);

Comparando con las etiquetas reales:

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y);

Examinando el resultado de forma gráfica, está claro que podríamos estar satisfechos con los resultados obtenidos, pero, en general, nos gustaría tener una evaluación cuantitativa. ¿Qué tal comparar nuestras etiquetas aprendidas con las etiquetas reales?

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

print('Porcentaje de precisión:', accuracy_score(y, labels))
print(confusion_matrix(y, labels))

In [ ]:
np.mean(y == labels)

In [ ]:
from sklearn.metrics import adjusted_rand_score

adjusted_rand_score(y, labels)

Una de las desventajas del K-means es que tenemos que especificar el número de *clusters*, cosa que a menudo no conocemos *a priori*. Por ejemplo, veamos que pasa si ponemos k=2 para el dataset sintético anterior:

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=42)
labels = kmeans.fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels);

In [ ]:
kmeans.cluster_centers_

#### El método del codo (*Elbow method*)

El método del codo es una regla general para encontrar el número óptimo de *clusters*. Para ello, analizamos la dispersión de los *clusters* (también llamada inercia o SSE, suma de las distancias de cada punto a su centroide más cercano) para distintos valores de k:

In [ ]:
distortions = []
for i in range(1, 11):
    km = KMeans(n_clusters=i, 
                random_state=0)
    km.fit(X)
    distortions.append(km.inertia_)

plt.plot(range(1, 11), distortions, marker='o')
plt.xlabel('Número de grupos')
plt.ylabel('Distorsión')
plt.show()

Después, tomamos el valor que lleva al pico del codo. Como podemos ver, ese valor será k=3 para este caso, lo que tiene sentido dado el conocimiento que tenemos del dataset.

**El agrupamiento siempre viene con suposiciones**: Un algoritmo de agrupamiento encuentra grupos haciendo suposiciones acerca de cómo debería agruparse los ejemplos. Cada algoritmo hace suposiciones distintas y la calidad e interpretabilidad de nuestros resultados dependerá de si estas suposiciones son correctas para nuestro objetivo. Para el K-means, el modelo subyacente supone que todos los grupos tienen la misma varianza, esférica (si usamos distancia Euclídea).

**En general, no hay ninguna garantía de que la estructura encontrada por un algoritmo de *clustering* sea aquello en lo que estás interesado**.


## Algunos métodos importantes de agrupamiento

A continuación, algunos de los algoritmos de agrupamiento más importantes: 

- `sklearn.cluster.KMeans`: <br/>
    El más simple aunque más efectivo algoritmo de agrupamiento. Hay que proporcionarle el número de grupos y asume que los datos están normalizados.
- `sklearn.cluster.MeanShift`: <br/>
    Puede encontrar *mejores* clusters que KMeans pero no es escalable para muchos ejemplos.
- `sklearn.cluster.DBSCAN`: <br/>
    Puede detectar grupos con formas irregulares, basándose en densidades, es decir, las regiones dispersas del espacio de entrada tiene más posibilidades de convertirse en fronteras entre *clusters*. También permite detectar *outliers* (datos que no pertenecen a ningún grupo.
- `sklearn.cluster.AffinityPropagation`: <br/>
    Algoritmo de agrupamiento basado en paso de mensajes entre puntos.
- `sklearn.cluster.SpectralClustering`: <br/>
    KMeans aplicado a la proyección de un grafo Laplaciano normalizado: encuentra cortes del grafo normalizado cuando la matriz de afinidad se interpreta como la matriz de adyacencia de un grafo.
- `sklearn.cluster.Ward`: <br/>
    Implementa clustering jerárquico basado en el algoritmo de Ward, una aproximación que minimiza la varianza intra-cluster. En cada paso, minimiza la suma de diferencias cuadradas internas de todos los puntos de todos los grupos (criterio de inercia).

Entre estos, el algoritmo de Ward, el SpectralClustering, el DBSCAN y el método de propagación de afinidad pueden trabajar también con matrices de similitud precomputadas.

<img src="figures/cluster_comparison.png" width="900">